In [11]:
from datetime import datetime
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import datapane as dp
import pandas as pd
import random
import sys
import time
import numpy
import math

#six decimal are enough for a decimeter precision
uav_latitude = 44.9908756
uav_longitude = 10.4523964

bts_latitude = 0
bts_longitude = 0

In [12]:
#Load BTS database
BTS_DATA = pd.read_csv('bts_windtre.csv', sep=';', header = None, prefix = ['operator','cell ID',])
BTS_DATA

,22288,26112256,0,-1,44.4131431,8.9324498,-1.1,B3 S1 DSS Genova Via Colombo,4
0,22299,26112256,0,-1,44.413143,8.932450,-1,B3 S1 DSS Genova Via Colombo,4
1,22288,26112257,0,-1,44.413143,8.932450,-1,B3 S2 DSS Genova Via Colombo,4
2,22299,26112257,0,-1,44.413143,8.932450,-1,B3 S2 DSS Genova Via Colombo,4
3,22288,26112262,0,-1,44.413143,8.932450,-1,B20 S1 DSS Genova Via Colombo,4
4,22299,26112262,0,-1,44.413143,8.932450,-1,B20 S1 DSS Genova Via Colombo,4
...,...,...,...,...,...,...,...,...,...
545716,22299,218683136,0,-1,38.025038,14.458089,-1,B3 S1 A 20 Galleria Caronia,4
545717,22288,218686464,0,-1,38.004634,14.316510,-1,B3 S1 Santo Stefano di Camastra A20 Galleria P...,4
545718,22299,218686464,0,-1,38.004634,14.316510,-1,B3 S1 Santo Stefano di Camastra A20 Galleria P...,4
545719,22288,218686465,0,-1,38.004634,14.316510,-1,B3 S2 Santo Stefano di Camastra A20 Galleria P...,4
